In [1]:
from OddoDownload import OdooDownloadBase,OdooDownloadCenco,OdooDownloadCorona,OdooDownloadTottus,OdooDownloadDimerc
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
conn_params_cenco = {
    'ODOO_USERNAME' : 'acceso@asalvo.cl',
    'ODOO_PASSWORD' : '1234',
    'ODOO_HOSTNAME' : 'sinergia-lab-cencorep-ambiente-test-9601418.dev.odoo.com',
    'ODOO_DATABASE' : 'sinergia-lab-cencorep-ambiente-test-9601418'
}

conn_params_corona = {
    'ODOO_USERNAME' : 'andres.romo@sinergiaindustrias.cl',
    'ODOO_PASSWORD' : 'clavefalsa',
    'ODOO_HOSTNAME' : 'repcorona-test.odoo.com',
    'ODOO_DATABASE' : 'repcorona-test'
}

conn_params_tottus = {
    'ODOO_USERNAME' : 'andres.romo@sinergiaindustrias.cl',
    'ODOO_PASSWORD' : 'clavefalsa',
    'ODOO_HOSTNAME' : 'rep-tottus-test.odoo.com',
    'ODOO_DATABASE' : 'rep-tottus-test'
}

conn_params_dimerc = {
    'ODOO_USERNAME' : 'andres.romo@sinergiaindustrias.cl',
    'ODOO_PASSWORD' : 'clavefalsa',
    'ODOO_HOSTNAME' : 'rep-dimerc-test.odoo.com',
    'ODOO_DATABASE' : 'rep-dimerc-test'
}

odoo_cenco = OdooDownloadCenco(conn_params_cenco)
odoo_corona = OdooDownloadCorona(conn_params_corona)
odoo_tottus = OdooDownloadTottus(conn_params_tottus)
odoo_dimerc = OdooDownloadDimerc(conn_params_dimerc)

In [4]:
periodo = 2023
unidad_negocio = 'DIMERC'


if unidad_negocio not in ['DIMERC','PRONOBEL','DIMEIGGS']:
    raise Exception('La unidad de negocio debe ser DIMERC, PRONOBEL o DIMEIGGS')
        
unidad_negocio_original = None

# =========================
# DESCARGAR TABLA DE VENTAS
# =========================

if unidad_negocio=='DIMERC':
    campos_ventas_totales = ['x_studio_unidades_vendidas_dimerc']
    header_ventas_totales = ['Unidades vendidas Dimerc']
    unidad_negocio = 'DIMERC'
    unidad_negocio_original = 'DIMERC'

elif unidad_negocio=='PRONOBEL':
    campos_ventas_totales = ['x_studio_unidades_vendidas_pronobel']
    header_ventas_totales = ['Unidades Vendidas Pronobel']
    unidad_negocio = 'DIMERC'
    unidad_negocio_original = 'PRONOBEL'

elif unidad_negocio=='DIMEIGGS':
    campos_ventas_totales = ['x_studio_unidades_vendidas_dimeiggs']
    header_ventas_totales = ['Unidades Vendidas Dimeiggs']
    unidad_negocio = 'DIMERC'
    unidad_negocio_original = 'DIMEIGGS'


# =========================
# DESCARGAR TABLA DE VENTAS
# =========================
modelo = 'x_unidades_vendidas'
filtros = ['&',('x_studio_unidad_de_negocio','=',unidad_negocio),('x_studio_periodo_1.x_name','=',periodo)]
campos = ['x_studio_producto','x_studio_descripcin_producto','x_studio_elementos_del_producto'] + campos_ventas_totales
campos = ['x_studio_producto'
]
header = ['Producto','Producto/Descripción','lista_elementos'] + header_ventas_totales 
campos_fk = ['x_studio_producto']

Descargando data desde x_productos
.
done
(False, True, True)


In [5]:
odoo_dimerc.resultadoBusqueda

,x_name,id,x_studio_stage_id,x_studio_levantamiento_asalvo
0,16780,424510,Completado,True
1,16781,402275,Completado,True
2,16787,102743,Completado,True
3,16792,427726,Completado,True
4,16794,402277,Completado,True
...,...,...,...,...
737,23362,80002445,No completado - Nuevo,False
738,23363,80002444,No completado - Nuevo,False
739,23364,80002630,No completado - Nuevo,False
740,23365,80002632,No completado - Nuevo,False
